In [ ]:
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Callable
from util import llm_call, extract_xml


def route(input: str, routes: Dict[str, str]) -> str:
    """Route input to specialized prompt using content classification."""
    # First determine appropriate route using LLM with chain-of-thought
    print(f"\nAvailable routes: {list(routes.keys())}")
    selector_prompt = f"""
    Analyze the input and select the most appropriate support team from these options: {list(routes.keys())}
    First explain your reasoning, then provide your selection in this XML format:

    <reasoning>
    Brief explanation of why this ticket should be routed to a specific team.
    Consider key terms, user intent, and urgency level.
    </reasoning>

    <selection>
    The chosen team name
    </selection>

    Input: {input}""".strip()
    
    route_response = llm_call(selector_prompt)
    reasoning = extract_xml(route_response, 'reasoning')
    route_key = extract_xml(route_response, 'selection').strip().lower()
    
    print("Routing Analysis:")
    print(reasoning)
    print(f"\nSelected route: {route_key}")
    
    # Process input with selected specialized prompt
    selected_prompt = routes[route_key]
    return llm_call(f"{selected_prompt}\nInput: {input}")



# Example 3: Route workflow for customer support ticket handling
# Route support tickets to appropriate teams based on content analysis

support_routes = {
    "billing": """You are a billing support specialist. Follow these guidelines:
    1. Always start with "Billing Support Response:"
    2. First acknowledge the specific billing issue
    3. Explain any charges or discrepancies clearly
    4. List concrete next steps with timeline
    5. End with payment options if relevant
    
    Keep responses professional but friendly.
    
    Input: """,
    
    "technical": """You are a technical support engineer. Follow these guidelines:
    1. Always start with "Technical Support Response:"
    2. List exact steps to resolve the issue
    3. Include system requirements if relevant
    4. Provide workarounds for common problems
    5. End with escalation path if needed
    
    Use clear, numbered steps and technical details.
    
    Input: """,
    
    "account": """You are an account security specialist. Follow these guidelines:
    1. Always start with "Account Support Response:"
    2. Prioritize account security and verification
    3. Provide clear steps for account recovery/changes
    4. Include security tips and warnings
    5. Set clear expectations for resolution time
    
    Maintain a serious, security-focused tone.
    
    Input: """,
    
    "product": """You are a product specialist. Follow these guidelines:
    1. Always start with "Product Support Response:"
    2. Focus on feature education and best practices
    3. Include specific examples of usage
    4. Link to relevant documentation sections
    5. Suggest related features that might help
    
    Be educational and encouraging in tone.
    
    Input: """
}

# Test with different support tickets
tickets = [
    """Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error. 
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to 
    submit a report by end of day.
    - John""",
    
    """Subject: Unexpected charge on my card
    Message: Hello, I just noticed a charge of $49.99 on my credit card from your company, but I thought
    I was on the $29.99 plan. Can you explain this charge and adjust it if it's a mistake?
    Thanks,
    Sarah""",
    
    """Subject: How to export data?
    Message: I need to export all my project data to Excel. I've looked through the docs but can't
    figure out how to do a bulk export. Is this possible? If so, could you walk me through the steps?
    Best regards,
    Mike"""
]

print("Processing support tickets...\n")
for i, ticket in enumerate(tickets, 1):
    print(f"\nTicket {i}:")
    print("-" * 40)
    print(ticket)
    print("\nResponse:")
    print("-" * 40)
    response = route(ticket, support_routes)
    print(response)

Processing support tickets...


Ticket 1:
----------------------------------------
Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error. 
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to 
    submit a report by end of day.
    - John

Response:
----------------------------------------

Available routes: ['billing', 'technical', 'account', 'product']
Routing Analysis:

    The input describes a user experiencing difficulty accessing their account due to an 'invalid password' error, despite being confident in the correctness of the password. The user's intent is to regain access to their account, which is urgent as they need to submit a report by the end of the day. The issue is related to account access and authentication, which falls under the domain of the 'account' support team. The 'account' team is responsible for handling login issues, password resets, and account recovery, making them the most appropriate choice for this scenario.
    

Selected route: account
**Account Support Response:**  

Thank you for reaching out, John. We understand the urgency of your situation and will assist you in regaining access to your account while ensuring its security.  

### **Steps to Regain Access:**  
1. **Password Reset:**  
   - Visit our [official password reset page](#) (do not use links from emails—type it manually).  
   - Enter your registered email address to receive a secure reset link.  
   - **Warning:** Never share this link or any verification codes with anyone.  

2. **Check for Unauthorized Activity:**  
   - After resetting your password, review recent login activity under **Account Settings > Security**.  
   - If you see unfamiliar devices/locations, enable **Two-Factor Authentication (2FA)** immediately.  

3. **Still Unable to Access?**  
   - If the issue persists, contact our **24/7 Security Team** at *security-support@example.com* with:  
     - Your full name  
     - Account email/username  
     - Any previous passwords (last 2 used) for verification  

### **Security Tips:**  
- **Never reuse passwords**—use a unique, strong password for this account.  
- **Beware of phishing emails**—we will never ask for your password via email or phone.  
- **Enable 2FA** for an extra layer of protection.  

### **Resolution Time:**  
- Password reset: **Immediate** (if email is accessible)  
- Manual account recovery: **24–48 hours** (requires verification)  

Act quickly but cautiously. If you suspect a breach, notify us immediately.  

**— Account Security Team**

Ticket 2:
----------------------------------------
Subject: Unexpected charge on my card
    Message: Hello, I just noticed a charge of $49.99 on my credit card from your company, but I thought
    I was on the $29.99 plan. Can you explain this charge and adjust it if it's a mistake?
    Thanks,
    Sarah

Response:
----------------------------------------

Available routes: ['billing', 'technical', 'account', 'product']
Routing Analysis:

The input mentions an unexpected charge on the user's credit card and a discrepancy between the expected plan ($29.99) and the actual charge ($49.99). The user is seeking clarification and potential adjustment of the charge. This issue is directly related to billing and payment, which falls under the purview of the 'billing' support team. Key terms like "charge," "credit card," and "plan" further indicate a billing-related inquiry. There is no mention of technical issues, account access problems, or product-related questions, so other teams are not relevant here.


Selected route: billing
**Billing Support Response:**  

Hi Sarah,  

Thank you for reaching out about the unexpected charge on your card. I completely understand your concern, and I’d be happy to clarify this for you.  

Upon reviewing your account, I see that the $49.99 charge reflects an upgrade to our Premium plan, which includes additional features beyond the $29.99 Basic plan. However, if this wasn’t authorized, we can certainly look into correcting it.  

**Next Steps:**  
1. I’ll verify whether the upgrade was intentional or accidental.  
2. If it was a mistake, I’ll process a refund or adjustment within **1–3 business days**.  
3. You’ll receive a confirmation email once the correction is made.  

**Payment Options (if needed):**  
If you’d like to keep the Premium plan, no further action is needed. Otherwise, we can revert you to the $29.99 plan—just let me know your preference!  

Please confirm if this sounds good, and I’ll assist right away.  

Best regards,  
[Your Name]  
Billing Support Team

Ticket 3:
----------------------------------------
Subject: How to export data?
    Message: I need to export all my project data to Excel. I've looked through the docs but can't
    figure out how to do a bulk export. Is this possible? If so, could you walk me through the steps?
    Best regards,
    Mike

Response:
----------------------------------------

Available routes: ['billing', 'technical', 'account', 'product']
Routing Analysis:

The input is about exporting project data to Excel, specifically asking for guidance on bulk export functionality. This involves understanding the product's features and how to use them, which falls under product support. The user mentions having checked the documentation but still needing assistance, indicating a need for detailed product knowledge rather than technical troubleshooting, account management, or billing issues.


Selected route: product
**Product Support Response:**  

Hi Mike,  

Thank you for reaching out! Exporting your project data to Excel is absolutely possible, and I’d be happy to walk you through the steps.  

### **How to Export Project Data to Excel:**  
1. **Navigate to the Projects Dashboard**: From the main menu, select **Projects** > **All Projects**.  
2. **Select Data for Export**: Use the checkboxes to choose specific projects or click "Select All" for a bulk export.  
3. **Click the Export Button**: Look for the **Export** option in the toolbar (typically represented by a download icon or labeled explicitly).  
4. **Choose Excel Format**: Select **Excel (.xlsx)** when prompted for file type.  
5. **Confirm and Download**: Review your selections and confirm—your file will download automatically.  

### **Example Use Case**:  
If you need to analyze task completion rates across multiple projects, exporting to Excel lets you leverage sorting, charts, or pivot tables for deeper insights.  

### **Helpful Documentation**:  
For more details, check out our [Exporting Projects Guide](https://example.com/export-projects-docs), which includes troubleshooting tips for large datasets.  

### **Related Features You Might Find Useful**:  
- **Scheduled Exports**: Automate recurring reports under **Settings > Data Automation**.  
- **Custom Fields in Exports**: Include additional columns by customizing your export template.  

Let me know if you run into any issues or need help tailoring the exported data further—we’re here to help!  

Best regards,  
[Your Name]  
Product Specialist